# Task 1: Binary Representations

Optional part: Calculate how many bits set in code, bits set is number of 1s

## 1. Implement Bitwise Left Rotation (`rotl`)

In [9]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotl(x, n=1):
    n %= 32  # Ensures n is within 0-31
    return ((x << n) & MASK_32) | ((x & MASK_32) >> (32 - n))

### Tests

In [28]:
if __name__ == "__main__":
    test_val = 0x12345678
    
    result_4 = rotl(test_val, 4)
    result_0 = rotl(test_val, 0)

    # Function to format binary output with leading zeros
    def to_bin_str(val):
        return bin(val)[2:].zfill(32)  # Ensures it's always 32 bits long

    print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})")
    print(f"rotl(4):    0x{result_4:X} ({to_bin_str(result_4)})")
    print(f"rotl(0):    0x{result_0:X} ({to_bin_str(result_0)})")

Original:   0x12345678 (00010010001101000101011001111000)
rotl(4):    0x23456781 (00100011010001010110011110000001)
rotl(0):    0x12345678 (00010010001101000101011001111000)


## 2. Implement Bitwise Right Rotation (`rotr`)


In [14]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotr(x, n=1):
    
    n %= 32  # Ensure n is within 0-31
    return ((x >> n) & MASK_32) | ((x << (32 - n)) & MASK_32)


### Tests

In [16]:
if __name__ == "__main__":
    test_val = 0x12345678 
    
    result_4 = rotr(test_val, 4)
    result_1 = rotr(test_val, 1)

    def to_bin_str(val):
        return bin(val)[2:].zfill(32)

    print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})")
    print(f"rotr(4):    0x{result_4:X} ({to_bin_str(result_4)})")
    print(f"rotr(1):    0x{result_1:X} ({to_bin_str(result_1)})")

Original:   0x12345678 (00010010001101000101011001111000)
rotr(4):    0x81234567 (10000001001000110100010101100111)
rotr(1):    0x91A2B3C (00001001000110100010101100111100)


## 3. Implement `ch(x, y, z)` Function


In [1]:
# Chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
def ch(x, y, z):
    return (x & y) ^ (~x & z)


### Tests

In [5]:
# x is diferent from both y and z
x_val = 0b1010 
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x is equal to y and diferent from z
x_val = 0b0000
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x is equal to z and diferent from y
x_val = 0b1111
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

ch(0b1010, 0b1100, 0b1111) = 0b1101
ch(0b1010, 0b1100, 0b1111) = 0b1111
ch(0b1010, 0b1100, 0b1111) = 0b1100


## 4. Implement `maj(x, y, z)` Function

In [6]:
# Implement a function that takes a majority vote of the bits in x, y, z.
# Each bit should be 1 if at least two of three inputs have 1 in that position.
def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

### Tests

In [12]:
# x has a mixture of 1s and 0s
x_val = 0b1010 
y_val = 0b1100  
z_val = 0b1111 
result = maj(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"maj(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x has all 1s
x_val = 0b1111 
y_val = 0b1100  
z_val = 0b1111 
result = maj(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"maj(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x has all 0s
x_val = 0b0000
y_val = 0b1100
z_val = 0b1111 
result = maj(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"maj(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

maj(0b1010, 0b1100, 0b1111) = 0b1110
maj(0b1010, 0b1100, 0b1111) = 0b1111
maj(0b1010, 0b1100, 0b1111) = 0b1000


# Task 2

## Convert C Hash Function to Python

In [14]:
# Converts a string to a hash value.
def hash_function(s):

    hashValue = 0
    # Hash value updated for each character in the string
    for char in s:
        # ord() gets ASCII value of the character
        hashValue = ord(char) + 31 * hashValue
    # Ensure the hash value is within 0-100
    return hashValue % 101

### Tests

In [15]:
test_strings = ["hello", "world", "python", "hash", "coding", "umbrella"]

for s in test_strings:
    print(f"hash({s}) = {hash_function(s)}")

hash(hello) = 17
hash(world) = 34
hash(python) = 91
hash(hash) = 15
hash(coding) = 73
hash(umbrella) = 78


## Expanded Hash Function

In [7]:
def hash_function_expanded(s):
    
    hashValue = 0
    print(f"\nHashing string '{s}':")
    print("-" * 77)

    for index, char in enumerate(s):
        ascii_value = ord(char)
        print(f"Step {index + 1}: char = '{char}' with an ASCII value of {ascii_value}, previous hash was {hashValue}")
        print(f"\thash = {ascii_value} + 31 * {hashValue}")
        hashValue = ascii_value + 31 * hashValue
        print(f"\tNew hash value after processing '{char}: {hashValue}")

    final_hash_value = hashValue % 101
    print(f"\nFinal hash value after modulo 101: {final_hash_value}")
    print("-" * 77)

    return final_hash_value

In [8]:
test_strings = ["hello", "world", "python", "hash", "coding", "umbrella", "rust"]

for s in test_strings:
    hash_function_expanded(s)



Hashing string 'hello':
-----------------------------------------------------------------------------
Step 1: char = 'h' with an ASCII value of 104, previous hash was 0
	hash = 104 + 31 * 0
	New hash value after processing 'h: 104
Step 2: char = 'e' with an ASCII value of 101, previous hash was 104
	hash = 101 + 31 * 104
	New hash value after processing 'e: 31301
Step 3: char = 'l' with an ASCII value of 108, previous hash was 31301
	hash = 108 + 31 * 31301
	New hash value after processing 'l: 9390408
Step 4: char = 'l' with an ASCII value of 108, previous hash was 9390408
	hash = 108 + 31 * 9390408
	New hash value after processing 'l: 2817122508
Step 5: char = 'o' with an ASCII value of 111, previous hash was 2817122508
	hash = 111 + 31 * 2817122508
	New hash value after processing 'o: 845136752511

Final hash value after modulo 101: 93
-----------------------------------------------------------------------------

Hashing string 'world':
----------------------------------------------

## Reasons For Using 31 and 101

**Why 31?**

- It is a **prime number**, this is important because it helps reduce the chance of **collisions** when hashing strings by distributing hash values more evenly accross the hash table. 

- A **collision** occurs when two different inputs produce the same hash value. 

- The multiplication using it makes the hash values less predictable and spread out over a wider range, minimising clustering.

- 31 is also a small number, reducing the likelyhood of integer overflow in C due to it having fixed sized integers (such as 32 or 64-bit). However, since this has been converted to Python this is no longer an issue as it automatically swtiches from fixed-sized intgers to arbitrary-precision integers when needed. It is still used here though to maintain consistancy between languages and to prevent performance slowdown which can still happen with extremely large numbers

- In C, multiplying by 31 can be optimised by compilers as a simple calculation it provides of 2^5 - 1 enables the efficient use of bit-shifting and subtraction instead of pure multiplication. Once again, however, this isn't a significant issue in Python as it handles operations like multiplication and bit-shifting at a higher level so there is no significant performance difference between them.

**Why 101?**

- It is also a prime number, in this case used in the modulo operation to limit the hash values to be within a specific range, in this case 0 to 100.

- Similar to 31, it being a prime number helps distribute the hash values more evenly across this range.

- Using 101 also helps prevent the clustering of hash values, this can occur if the modulo base has common factors with the data. Examples of these factors are if the data contains patterns divisible by the modulo base. 101 doesn't share common factors with most data patterns, helping to greatly negate this risk.

# Task 3

# Task 4

# Task 5

# Task 6

# Task 7

# Task 8